In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 🚗 Preuve de Concept : Segmentation Sémantique pour Véhicules Autonomes

Ce notebook implémente un pipeline de segmentation sémantique comparant une architecture classique (**U-Net**) et des modèles d'état de l'art (**YOLO**).

## 📑 Plan du Notebook
1. **Installation et Importations** : Préparation de l'environnement de travail.
2. **Configuration et Données** : Mapping des classes Cityscapes et configuration globale.
3. **Augmentation de Données** : Implémentation de la technique **CutMix**.
4. **Dataset et Métriques** : Préparation du chargeur de données et des fonctions d'évaluation.
5. **Analyse Exploratoire (EDA)** : Visualisation de la distribution et exemples.
6. **Entraînement** : Fonctions d'entraînement pour U-Net et YOLO.
7. **Visualisation** : Outils de génération de masques colorés.
8. **Exécution et Comparaison** : Lancement des entraînements et affichage des résultats.
9. **Optimisation et Analyse** : Optimisation du meilleur modèle et analyse approfondie.
10. **Conclusion** : Synthèse des résultats.

In [ ]:
# Installation des librairies nécessaires
!pip install torch torchvision opencv-python pandas ultralytics segmentation-models-pytorch albumentations

import os
import time
import cv2
import yaml
import glob
import random
import requests
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
from tensorflow.keras import layers, models, applications
from torch.utils.data import Dataset, DataLoader, random_split
import segmentation_models_pytorch as smp
from ultralytics import YOLO
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from tqdm import tqdm
from sklearn.metrics import accuracy_score, jaccard_score, confusion_matrix, ConfusionMatrixDisplay
import json

## 2. Configuration et Préparation des Données
Nous définissons ici le mapping des classes pour regrouper les 34 classes originales de Cityscapes en 8 catégories principales (Véhicules, Humains, Route, etc.).

In [ ]:
# --- Mapping des classes Cityscapes (8 Classes) ---
# Inclut la classe 0 (Route/Flat).

cityscapes_mapping = {
    0: 7, 1: 7, 2: 7, 3: 7, 4: 7, 5: 7, 6: 7, # void
    7: 0, 8: 0, 9: 0, 10: 0, # road
    11: 3, 12: 3, 13: 3, 14: 3, 15: 3, 16: 3, # construction
    17: 4, 18: 4, 19: 4, 20: 4, # object
    21: 5, 22: 5, # nature
    23: 6, # sky
    24: 1, 25: 1, # human
    26: 2, 27: 2, 28: 2, 29: 2, 30: 2, 31: 2, 32: 2, 33: 2 # vehicle
}

# Groupes originaux : 0=Road, 1=Human, 2=Vehicle, 3=Const., 4=Object, 5=Nature, 6=Sky, 7=Void
# Nouveau mapping : 0..7 -> 0..7
id_to_group = np.full(256, 255, dtype=np.uint8) # Default to ignore for unmapped Cityscapes IDs
for id_val, group_id in cityscapes_mapping.items():
    id_to_group[id_val] = group_id # Direct mapping to new 0-7 classes

# Couleurs pour la visualisation (RGB)
original_colors = [[128, 64, 128], [220, 20, 60], [0, 0, 142], [70, 70, 70], [220, 220, 0], [107, 142, 35], [70, 130, 180], [0, 0, 0]]
group_colors = original_colors # Use all original colors
NUM_CLASSES = 8 # Now 8 classes

# --- Configuration Globale ---
BASE_PATH = '/content/drive/MyDrive/P10_Developpez une preuve de concept'
DATA_DIR = os.path.join(BASE_PATH, 'data')

CONFIG = {
    'seed': 42,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'img_size': (256, 256),
    'test_images_dir': os.path.join(DATA_DIR, 'images_tests'),
    'output_dir': os.path.join(BASE_PATH, 'results'),
    'yolo_runs_dir': os.path.join(BASE_PATH, 'yolo_runs'),
    'mini_unet': {
        'img_dir': os.path.join(DATA_DIR, 'img'),
        'lbl_dir': os.path.join(DATA_DIR, 'label'),
        'lr': 0.0001,
        'batch_size': 8,
        'epochs': 20,
        'backbone': 'vgg16',
        'use_cutmix': True
    },
    'yolo_models': [
        {'name': 'YOLOv8n', 'version': 'yolov8n-seg.pt', 'data': os.path.join(DATA_DIR, 'yoloautodrivesegmentation.v5-v5.yolov8/data.yaml')},
        {'name': 'YOLOv8s', 'version': 'yolov8s-seg.pt', 'data': os.path.join(DATA_DIR, 'yoloautodrivesegmentation.v5-v5.yolov8/data.yaml')},
        {'name': 'YOLOv8m', 'version': 'yolov8m-seg.pt', 'data': os.path.join(DATA_DIR, 'yoloautodrivesegmentation.v5-v5.yolov8/data.yaml')},
        {'name': 'YOLOv9', 'version': 'yolov9c-seg.pt', 'data': os.path.join(DATA_DIR, 'yoloautodrivesegmentation.v5-v5.yolov9/data.yaml')},
        {'name': 'YOLOv9e', 'version': 'yolov9e-seg.pt', 'data': os.path.join(DATA_DIR, 'yoloautodrivesegmentation.v5-v5.yolov9/data.yaml')},
        {'name': 'YOLOv11n', 'version': 'yolo11n-seg.pt', 'data': os.path.join(DATA_DIR, 'yoloautodrivesegmentation.v5-v5.yolov11/data.yaml')},
    ]
}

def set_seed(seed):
    """Fixe la graine aléatoire pour la reproductibilité."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

set_seed(CONFIG['seed'])
os.makedirs(CONFIG['output_dir'], exist_ok=True)

# --- Correction Dynamique du YAML (Fix Path Error) ---
def fix_yaml_paths(yaml_path):
    """Corrige les chemins dans le fichier data.yaml pour pointer vers DATA_DIR."""
    if not os.path.exists(yaml_path):
        return

    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)

    # On définit le dossier parent du fichier yaml comme racine du dataset
    dataset_root = os.path.dirname(yaml_path)

    # Mise à jour des clés standards de YOLO
    data['path'] = dataset_root
    data['train'] = 'train/images'
    data['val'] = 'valid/images'
    if 'test' in data:
        data['test'] = 'test/images'

    with open(yaml_path, 'w') as f:
        yaml.dump(data, f, default_flow_style=False)
    print(f"🔧 YAML corrigé : {yaml_path}")

## 3. Augmentation de Données : CutMix
La technique **CutMix** consiste à découper une partie d'une image et à la coller sur une autre, tout en mélangeant les étiquettes (labels) proportionnellement. Cela force le modèle à ne pas se fier uniquement à des caractéristiques locales évidentes.

In [ ]:
# --- Implémentation CutMix ---
def apply_cutmix(images, masks, alpha=1.0):
    """Applique l'augmentation CutMix sur un batch d'images et de masques."""
    indices = torch.randperm(images.size(0))
    shuffled_images, shuffled_masks = images[indices], masks[indices]
    lam = np.random.beta(alpha, alpha)
    W, H = images.size(2), images.size(3)
    cut_rat = np.sqrt(1. - lam)
    cut_w, cut_h = np.int32(W * cut_rat), np.int32(H * cut_rat)
    cx, cy = np.random.randint(W), np.random.randint(H)
    bbx1, bby1 = np.clip(cx - cut_w // 2, 0, W), np.clip(cy - cut_h // 2, 0, H)
    bbx2, bby2 = np.clip(cx + cut_w // 2, 0, W), np.clip(cy + cut_h // 2, 0, H)
    images[:, :, bbx1:bbx2, bby1:bby2] = shuffled_images[:, :, bbx1:bbx2, bby1:bby2]
    masks[:, bbx1:bbx2, bby1:bby2] = shuffled_masks[:, bbx1:bbx2, bby1:bby2]
    return images, masks

## 4. Dataset et Métriques
Nous définissons ici la classe `SegmentationDataset` pour charger les images et les masques, ainsi que les métriques d'évaluation (Accuracy et mIoU).

In [ ]:
# --- Dataset & Metrics ---
def calculate_pixel_metrics(y_true, y_pred):
    """Calcule l'accuracy pixel par pixel et l'IoU moyen (mIoU) en ignorant l'index 255."""
    y_true, y_pred = y_true.flatten(), y_pred.flatten()

    # Filtrer les pixels ignorés (255)
    mask = y_true != 255
    y_true = y_true[mask]
    y_pred = y_pred[mask]

    if len(y_true) == 0:
        return 0.0, 0.0

    acc = accuracy_score(y_true, y_pred)
    iou = jaccard_score(y_true, y_pred, average='macro', zero_division=1.0)
    return acc, iou

class SegmentationDataset(Dataset):
    def __init__(self, img_dir, mask_dir, transform=None):
        self.img_dir, self.mask_dir = img_dir, mask_dir
        self.images = sorted([f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png'))])
        self.transform = transform
    def __len__(self): return len(self.images)
    def __getitem__(self, idx):
        img_p = os.path.join(self.img_dir, self.images[idx])
        mask_p = os.path.join(self.mask_dir, self.images[idx])
        image = cv2.cvtColor(cv2.imread(img_p), cv2.COLOR_BGR2RGB)
        # Conversion du masque en niveaux de gris vers les groupes d'ID
        # id_to_group gère maintenant le remapping et l'ignore index (255)
        mask = id_to_group[cv2.imread(mask_p, cv2.IMREAD_GRAYSCALE)]
        if self.transform:
            aug = self.transform(image=image, mask=mask)
            image, mask = aug['image'], aug['mask']
        return image, mask.long()

## 5. Analyse Exploratoire des Données (EDA)
Avant de lancer l'entraînement, nous visualisons la distribution des classes et quelques exemples d'images avec leurs masques pour vérifier la qualité des données.

In [ ]:
# --- Analyse Exploratoire (EDA) ---
print("\n📊 Lancement de l'Analyse Exploratoire (EDA)...")

def perform_eda():
    img_dir, lbl_dir = CONFIG['mini_unet']['img_dir'], CONFIG['mini_unet']['lbl_dir']
    if not os.path.exists(img_dir):
        print("⚠️ Dossier d'images introuvable. EDA ignorée.")
        return

    # Dataset pour EDA (Resize + ToTensor, pas de normalisation pour affichage direct)
    eda_transform = A.Compose([A.Resize(*CONFIG['img_size']), ToTensorV2()])
    ds = SegmentationDataset(img_dir, lbl_dir, transform=eda_transform)

    if len(ds) == 0:
        print("⚠️ Dataset vide.")
        return

    # 1. Distribution des classes (sur l'ensemble du dataset)
    print(f"   Calcul de la distribution des classes sur l'ensemble du dataset ({len(ds)} images)...")

    # Récupération directe des chemins de masques pour éviter de charger les images (plus rapide)
    mask_files = sorted(glob.glob(os.path.join(lbl_dir, '*.*')))

    total_class_counts = np.zeros(NUM_CLASSES, dtype=np.int64)
    total_pixels_overall = 0

    for mask_path in tqdm(mask_files, desc="Comptage des pixels"):
        # Lecture en niveaux de gris
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None: continue

        # Application du mapping (0-33 -> 0-7)
        mask_mapped = id_to_group[mask]

        unique, counts = np.unique(mask_mapped, return_counts=True)
        for u, c in zip(unique, counts):
            if u < NUM_CLASSES: # This ensures only mapped classes are counted, 255 (ignore) is excluded
                total_class_counts[u] += c

        total_pixels_overall += mask.size

    # Calcul des pourcentages
    class_distribution = {i: (count / total_pixels_overall * 100) if total_pixels_overall > 0 else 0
                          for i, count in enumerate(total_class_counts)}

    print("\n📊 Distribution des classes (en % de pixels):")
    # Correct group_names for 8 classes (0-7)
    group_names = ['Flat', 'Human', 'Vehicle', 'Construction', 'Object', 'Nature', 'Sky', 'Void']
    for i, pct in class_distribution.items():
        print(f"  {group_names[i]}: {pct:.2f}%")

    # Création et sauvegarde du DataFrame de distribution
    df_dist = pd.DataFrame({
        'Classe': group_names,
        'Nombre de Pixels': total_class_counts,
        'Pourcentage (%)': [class_distribution[i] for i in range(NUM_CLASSES)]
    })
    dist_csv_path = os.path.join(CONFIG['output_dir'], 'class_distribution.csv')
    df_dist.to_csv(dist_csv_path, index=False)
    print(f"\n💾 Distribution des classes sauvegardée dans : {dist_csv_path}")

    # Calcul des poids de classes (Balanced: N_total / (N_classes * N_count))
    # On ajoute un epsilon pour éviter la division par zéro
    epsilon = 1e-6
    class_weights = total_pixels_overall / (NUM_CLASSES * (total_class_counts + epsilon))

    # Sauvegarde des poids
    weights_path = os.path.join(CONFIG['output_dir'], 'class_weights.json')
    with open(weights_path, 'w') as f:
        json.dump(class_weights.tolist(), f, indent=4)
    print(f"\n⚖️ Poids des classes calculés et sauvegardés dans : {weights_path}")
    for i, w in enumerate(class_weights):
        print(f"  - {group_names[i]}: {w:.4f}")

    # Affichage Bar Chart
    plt.figure(figsize=(10, 4))
    # Use all group_colors (which now contains 8 colors)
    norm_colors = [np.array(c)/255. for c in group_colors]
    bars = plt.bar(range(NUM_CLASSES), total_class_counts, color=norm_colors, edgecolor='black')
    plt.title("Distribution des Classes (Pixels)")
    plt.xlabel("ID Classe (Groupe)")
    plt.ylabel("Nombre de Pixels")
    plt.xticks(range(NUM_CLASSES), group_names, rotation=45, ha='right') # Add rotation for long names
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout() # Ensure labels fit
    plt.show()

    # 2. Visualisation Exemples
    print("   Exemples d'images et masques...")
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    sample_indices = np.random.choice(len(ds), 4, replace=False)

    for i, idx in enumerate(sample_indices):
        img, mask = ds[idx]
        # Image: Tensor (C,H,W) -> Numpy (H,W,C)
        img_np = img.permute(1, 2, 0).numpy()

        # Masque: Tensor (H,W) -> Color Map
        mask_np = mask.numpy()
        color_mask = np.zeros((*mask_np.shape, 3), dtype=np.uint8)
        for cid, color in enumerate(group_colors): # This will loop through 8 colors
            color_mask[mask_np == cid] = color

        axes[0, i].imshow(img_np)
        axes[0, i].set_title(f"Image {idx}")
        axes[0, i].axis('off')

        axes[1, i].imshow(color_mask)
        axes[1, i].set_title(f"Masque {idx}")
        axes[1, i].axis('off')

    plt.tight_layout()
    plt.show()

perform_eda()

## 6. Entraînement des Modèles
Cette section contient les fonctions pour entraîner le modèle **Mini-Unet** (avec PyTorch standard) et les modèles **YOLO** (via la librairie Ultralytics).

In [ ]:
# --- Entraînement ---
def cutmix_data(img1, mask1, img2, mask2, img_size):
    """Implémentation de CutMix pour TensorFlow."""
    alpha = 1.0
    lam = tf.random.gamma(shape=[], alpha=alpha, dtype=tf.float32)
    
    H, W = img_size[0], img_size[1]
    
    cut_rat = tf.sqrt(1.0 - lam)
    cut_w = tf.cast(tf.cast(W, tf.float32) * cut_rat, tf.int32)
    cut_h = tf.cast(tf.cast(H, tf.float32) * cut_rat, tf.int32)
    
    cut_x = tf.random.uniform([], 0, W, dtype=tf.int32)
    cut_y = tf.random.uniform([], 0, H, dtype=tf.int32)
    
    bbx1 = tf.clip_by_value(cut_x - cut_w // 2, 0, W)
    bby1 = tf.clip_by_value(cut_y - cut_h // 2, 0, H)
    bbx2 = tf.clip_by_value(cut_x + cut_w // 2, 0, W)
    bby2 = tf.clip_by_value(cut_y + cut_h // 2, 0, H)
    
    # Création du masque binaire pour la zone découpée
    indices = tf.meshgrid(tf.range(H), tf.range(W), indexing='ij')
    mask_cut = (indices[0] >= bby1) & (indices[0] < bby2) & (indices[1] >= bbx1) & (indices[1] < bbx2)
    mask_cut = tf.cast(tf.expand_dims(mask_cut, -1), tf.float32)
    
    img = img1 * (1 - mask_cut) + img2 * mask_cut
    mask = mask1 * (1 - mask_cut) + mask2 * mask_cut
    
    return img, mask

def mixup_data(img1, mask1, img2, mask2):
    """Implémentation de MixUp pour TensorFlow."""
    alpha = 0.2
    lam = tf.random.gamma(shape=[], alpha=alpha, dtype=tf.float32)
    lam = tf.reshape(lam, [])
    
    img = img1 * lam + img2 * (1 - lam)
    mask = mask1 * lam + mask2 * (1 - lam)
    return img, mask

def TFDataGenerator(image_paths, mask_paths, batch_size, img_size, augmentation_type='none', albumentations_transform=None):
    """Crée un dataset TensorFlow à partir de listes de chemins avec augmentation configurable."""
    path_dataset = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))

    def load_and_preprocess(img_path_tensor, mask_path_tensor):
        img = tf.io.read_file(img_path_tensor)
        img = tf.image.decode_png(img, channels=3)
        img = tf.image.resize(img, img_size, method='bilinear')
        img = tf.cast(img, tf.float32) / 255.0

        mask = tf.io.read_file(mask_path_tensor)
        mask = tf.image.decode_png(mask, channels=1)
        mask = tf.image.resize(mask, img_size, method='nearest')
        mask = tf.cast(mask, tf.int32)

        flat_mask = tf.reshape(mask, [-1])
        mapped_mask = tf.gather(tf.constant(id_to_group, dtype=tf.int32), flat_mask)
        mask = tf.reshape(mapped_mask, tf.shape(mask))
        # Explicitly add channel dimension if it's not already there
        if mask.shape.ndims == 2:
             mask = tf.expand_dims(mask, axis=-1)
        # Ensure mask has a channel dimension of 1
        elif mask.shape.ndims == 3 and mask.shape[-1] != 1:
             mask = tf.expand_dims(tf.squeeze(mask, axis=-1), axis=-1) # Squeeze and re-add if channel dim exists but is not 1
        # Ensure the shape is known
        mask = tf.ensure_shape(mask, [img_size[0], img_size[1], 1])
        # Cast mask to float32 for augmentation operations
        mask = tf.cast(mask, tf.float32)
        return img, mask

    dataset = path_dataset.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)

    # Mettre en cache le dataset après le chargement et le redimensionnement pour accélérer les époques suivantes
    dataset = dataset.cache()

    if augmentation_type in ['cutmix', 'mixup']:
        dataset_zipped = tf.data.Dataset.zip((dataset, dataset.shuffle(batch_size * 10)))
        if augmentation_type == 'cutmix':
             dataset = dataset_zipped.map(lambda ds1, ds2: cutmix_data(ds1[0], ds1[1], ds2[0], ds2[1], img_size), num_parallel_calls=tf.data.AUTOTUNE)
        elif augmentation_type == 'mixup':
             dataset = dataset_zipped.map(lambda ds1, ds2: mixup_data(ds1[0], ds1[1], ds2[0], ds2[1]), num_parallel_calls=tf.data.AUTOTUNE)

    # Cast the mask back to int32 and squeeze the last dimension after any augmentation steps
    dataset = dataset.map(lambda x, y: (x, tf.squeeze(tf.cast(y, tf.int32), axis=-1)), num_parallel_calls=tf.data.AUTOTUNE)
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

def get_unet_vgg16_model(input_shape, num_classes):
    """Construit un modèle U-Net avec encodeur VGG16 en utilisant Keras."""
    inputs = layers.Input(shape=input_shape)
    
    # Encoder (VGG16)
    base_model = applications.VGG16(include_top=False, weights='imagenet', input_tensor=inputs)
    
    # Skip connections
    s1 = base_model.get_layer("block1_conv2").output
    s2 = base_model.get_layer("block2_conv2").output
    s3 = base_model.get_layer("block3_conv3").output
    s4 = base_model.get_layer("block4_conv3").output
    bridge = base_model.get_layer("block5_conv3").output

    # Decoder
    def decoder_block(input_tensor, skip_tensor, filters):
        x = layers.Conv2DTranspose(filters, (2, 2), strides=(2, 2), padding="same")(input_tensor)
        x = layers.Concatenate()([x, skip_tensor])
        x = layers.Conv2D(filters, (3, 3), padding="same", activation="relu")(x)
        x = layers.Conv2D(filters, (3, 3), padding="same", activation="relu")(x)
        return x

    d1 = decoder_block(bridge, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = layers.Conv2D(num_classes, (1, 1), activation="softmax")(d4)
    
    model = models.Model(inputs, outputs, name="Mini-Unet-VGG16")
    return model

def train_mini_unet():
    """Entraîne un modèle U-Net avec backbone VGG16 (TensorFlow/Keras)."""
    print(f"\n🚀 Entraînement Mini-Unet (TF) - {CONFIG['img_size']} ({NUM_CLASSES} Classes)...")
    img_dir, lbl_dir = CONFIG['mini_unet']['img_dir'], CONFIG['mini_unet']['lbl_dir']
    if not os.path.exists(img_dir): return None, {}

    # Récupération des chemins
    image_paths = sorted(glob.glob(os.path.join(img_dir, '*.*')))
    mask_paths = sorted(glob.glob(os.path.join(lbl_dir, '*.*')))
    
    # Split Train/Val (80/20)
    split_idx = int(0.8 * len(image_paths))
    train_img_paths, val_img_paths = image_paths[:split_idx], image_paths[split_idx:]
    train_mask_paths, val_mask_paths = mask_paths[:split_idx], mask_paths[split_idx:]

    # Configuration de l'augmentation
    aug_type = 'cutmix' if CONFIG['mini_unet']['use_cutmix'] else 'none'

    # Création des datasets TF
    train_ds = TFDataGenerator(train_img_paths, train_mask_paths, 
                               CONFIG['mini_unet']['batch_size'], 
                               CONFIG['img_size'], 
                               augmentation_type=aug_type)
    
    val_ds = TFDataGenerator(val_img_paths, val_mask_paths, 
                             CONFIG['mini_unet']['batch_size'], 
                             CONFIG['img_size'], 
                             augmentation_type='none')

    # Création du modèle
    model = get_unet_vgg16_model(input_shape=(CONFIG['img_size'][0], CONFIG['img_size'][1], 3), num_classes=NUM_CLASSES)
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=CONFIG['mini_unet']['lr']),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])

    # Entraînement
    start = time.time()
    history = model.fit(train_ds, validation_data=val_ds, epochs=CONFIG['mini_unet']['epochs'], verbose=1)

    # Sauvegarde des courbes d'entraînement
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss'); plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title('Loss'); plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Acc'); plt.plot(history.history['val_accuracy'], label='Val Acc')
    plt.title('Accuracy'); plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(CONFIG['output_dir'], 'mini_unet_curves.png'))
    plt.close()
    
    # Évaluation et Métriques
    # Calcul manuel du mIoU sur le dataset de validation pour être comparable
    all_preds, all_trues = [], []
    
    # Pour l'inférence, on itère sur le dataset de validation
    # Note: val_ds est batché, on dé-batch pour l'évaluation métrique précise ou on prédit par batch
    print("Calcul des métriques finales...")
    for imgs, msks in val_ds:
        preds = model.predict(imgs, verbose=0)
        pred_masks = tf.argmax(preds, axis=-1).numpy()
        true_masks = msks.numpy()
        
        all_preds.extend(pred_masks.flatten())
        all_trues.extend(true_masks.flatten())

    inf_t = ((time.time()-start)/len(val_img_paths))*1000 # Temps moyen par image (approximatif global)
    
    # Sauvegarde de la matrice de confusion
    cm = confusion_matrix(all_trues, all_preds, labels=range(NUM_CLASSES))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Flat', 'Human', 'Vehicle', 'Constr.', 'Object', 'Nature', 'Sky', 'Void'])
    fig, ax = plt.subplots(figsize=(10, 10))
    disp.plot(ax=ax, cmap='Blues', xticks_rotation='vertical', values_format='d')
    plt.savefig(os.path.join(CONFIG['output_dir'], 'mini_unet_cm.png'))
    plt.close()

    acc, iou = calculate_pixel_metrics(np.array(all_trues), np.array(all_preds))
    return model, {'Model': 'Mini-Unet (TF)', 'mIoU': iou, 'Inférence (ms)': inf_t}

def train_yolo_model(y_cfg):
    """Entraîne un modèle YOLO pour la segmentation."""
    print(f"\n🚀 Entraînement {y_cfg['name']} (Inclut toutes les classes 0-{NUM_CLASSES-1})...")
    # Correction du YAML avant de charger le modèle
    fix_yaml_paths(y_cfg['data'])

    model = YOLO(y_cfg['version'])
    model.train(
        data=y_cfg['data'],
        epochs=CONFIG['mini_unet']['epochs'],
        imgsz=CONFIG['img_size'][0],
        project=CONFIG['yolo_runs_dir'],
        name=y_cfg['name'],
        # Inclut toutes les classes (0 à NUM_CLASSES-1)
        classes=list(range(NUM_CLASSES)), # Changed this line
        exist_ok=True,
        verbose=False
    )
    # Validation sur les mêmes classes
    metrics = model.val(imgsz=CONFIG['img_size'][0], verbose=False)
    miou = metrics.seg.map if hasattr(metrics, 'seg') else metrics.box.map
    return model, {'Model': y_cfg['name'], 'mIoU': miou, 'Inférence (ms)': metrics.speed.get('inference', 0.0)}

## 7. Visualisation
Fonction utilitaire pour superposer les masques prédits sur les images originales.

In [ ]:
# --- Visualisation ---

def get_prediction_viz(model, model_name, img_path):
    """Génère une visualisation de la prédiction (Overlay)."""
    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]

    if isinstance(model, YOLO):
        # Inférence pour YOLO
        results = model.predict(img_path, imgsz=CONFIG['img_size'][0], conf=0.25, verbose=False)
        return cv2.resize(cv2.cvtColor(results[0].plot(), cv2.COLOR_BGR2RGB), (w, h))
    elif isinstance(model, torch.nn.Module):
        # Inférence pour U-Net
        model.eval()
        t = A.Compose([A.Resize(*CONFIG['img_size']), A.Normalize(), ToTensorV2()])
        inp = t(image=img)['image'].unsqueeze(0).to(CONFIG['device'])
        with torch.no_grad():
            mask = torch.argmax(model(inp), dim=1).squeeze().cpu().numpy().astype(np.uint8)
        mask_r = cv2.resize(mask, (w, h), interpolation=cv2.INTER_NEAREST)
        color_m = np.zeros_like(img)
        for i, color in enumerate(group_colors): color_m[mask_r == i] = color
        return cv2.addWeighted(img, 0.5, color_m, 0.5, 0)
    elif isinstance(model, (tf.keras.Model, tf.Module)):
        # Inférence pour TensorFlow/Keras
        img_tensor = tf.io.read_file(img_path)
        img_tensor = tf.image.decode_png(img_tensor, channels=3)
        img_tensor = tf.image.resize(img_tensor, CONFIG['img_size'])
        img_tensor = tf.cast(img_tensor, tf.float32) / 255.0
        img_tensor = tf.expand_dims(img_tensor, axis=0)
        
        pred = model.predict(img_tensor, verbose=0)
        mask = tf.argmax(pred, axis=-1)
        mask = mask[0].numpy().astype(np.uint8)
        
        mask_r = cv2.resize(mask, (w, h), interpolation=cv2.INTER_NEAREST)
        color_m = np.zeros_like(img)
        for i, color in enumerate(group_colors): color_m[mask_r == i] = color
        return cv2.addWeighted(img, 0.5, color_m, 0.5, 0)

def generate_pdf_report(df, output_dir, yolo_runs_dir):
    """Génère un rapport PDF complet avec tableaux et graphiques."""
    print("\n📄 Génération du rapport PDF...")
    pdf_path = os.path.join(output_dir, 'report.pdf')
    
    with PdfPages(pdf_path) as pdf:
        # Page 1: Tableau des résultats
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.axis('tight'); ax.axis('off')
        table_data = [df.columns.values.tolist()] + df.values.tolist()
        table = ax.table(cellText=table_data, colLabels=None, cellLoc='center', loc='center')
        table.auto_set_font_size(False); table.set_fontsize(10)
        plt.title("Tableau Comparatif des Performances")
        pdf.savefig(fig); plt.close()
        
        # Pages par modèle
        for _, row in df.iterrows():
            model_name = row['Model']
            fig = plt.figure(figsize=(11, 8))
            plt.axis('off'); plt.title(f"Rapport: {model_name}", fontsize=16, fontweight='bold')
            
            if 'YOLO' in model_name:
                run_path = os.path.join(yolo_runs_dir, model_name)
                curves_path = os.path.join(run_path, 'results.png')
                cm_path = os.path.join(run_path, 'confusion_matrix_normalized.png')
                if not os.path.exists(cm_path): cm_path = os.path.join(run_path, 'confusion_matrix.png')
            else:
                curves_path = os.path.join(output_dir, 'mini_unet_curves.png')
                cm_path = os.path.join(output_dir, 'mini_unet_cm.png')
                
            if curves_path and os.path.exists(curves_path):
                ax1 = fig.add_axes([0.1, 0.5, 0.8, 0.4]); ax1.imshow(plt.imread(curves_path)); ax1.axis('off'); ax1.set_title("Courbes d'entraînement")
            if cm_path and os.path.exists(cm_path):
                ax2 = fig.add_axes([0.1, 0.05, 0.8, 0.4]); ax2.imshow(plt.imread(cm_path)); ax2.axis('off'); ax2.set_title("Matrice de Confusion")
            pdf.savefig(fig); plt.close()
    print(f"✅ Rapport PDF sauvegardé : {pdf_path}")

## 8. Exécution Principale et Comparaison
Lancement du pipeline complet : entraînement de tous les modèles, comparaison des performances et affichage d'une galerie de résultats.

In [ ]:
results_list, trained_models = [], {}

# 1. Entraînement Mini-Unet
m_unet, met_u = train_mini_unet()
if m_unet:
    trained_models['Mini-Unet'] = m_unet
    results_list.append(met_u)

# 2. Entraînement Modèles YOLO (V8, V9, V11)
for yolo_cfg in CONFIG['yolo_models']:
    try:
        m_yolo, met_y = train_yolo_model(yolo_cfg)
        trained_models[yolo_cfg['name']] = m_yolo
        results_list.append(met_y)
    except Exception as e:
        print(f"⚠️ Erreur {yolo_cfg['name']}: {e}")

if results_list:
    # Création du DataFrame de résultats
    df = pd.DataFrame(results_list)
    print("\n📊 RÉSULTATS FINAUX (256x256 + CutMix) :")
    print(df.to_string(index=False))
    df.to_csv(os.path.join(CONFIG['output_dir'], 'comparison_results.csv'), index=False)

    # 3. Rapport et Export
    print("\n📊 TABLEAU RÉCAPITULATIF DES PERFORMANCES (Taille: 256x256) :")
    print("=" * 60)
    print(df.to_string(index=False))
    print("=" * 60)
    csv_path = os.path.join(CONFIG['output_dir'], 'final_model_comparison_256.csv')
    df.to_csv(csv_path, index=False)

    # Génération du rapport PDF
    generate_pdf_report(df, CONFIG['output_dir'], CONFIG['yolo_runs_dir'])

# 4. Galerie de Comparaison
test_files = glob.glob(os.path.join(CONFIG['test_images_dir'], '*.*'))
if test_files and trained_models:
    print(f"\n🎨 Génération des galeries de comparaison...")
    for img_p in test_files[:3]:
        fname = os.path.basename(img_p)
        n_models = len(trained_models)
        fig, axes = plt.subplots(1, n_models + 1, figsize=(5 * (n_models + 1), 5))

        # Image Originale
        axes[0].imshow(cv2.cvtColor(cv2.imread(img_p), cv2.COLOR_BGR2RGB))
        axes[0].set_title("Original")
        axes[0].axis('off')

        # Prédictions des modèles
        for i, (name, m_obj) in enumerate(trained_models.items()):
            axes[i+1].imshow(get_prediction_viz(m_obj, name, img_p))
            axes[i+1].set_title(name)
            axes[i+1].axis('off')

        plt.tight_layout()
        plt.savefig(os.path.join(CONFIG['output_dir'], f"comparison_256_{fname}"))
        plt.show()

In [ ]:
# 5. Optimisation du Meilleur Modèle YOLO
print("\n🔧 Recherche et Optimisation du meilleur modèle YOLO...")
yolo_results = df[df['Model'].str.contains('YOLO')]
if not yolo_results.empty:
    best_yolo_idx = yolo_results['mIoU'].idxmax()
    best_yolo_name = yolo_results.loc[best_yolo_idx, 'Model']
    print(f"🏆 Meilleur modèle YOLO identifié : {best_yolo_name}")

    if best_yolo_name in trained_models:
        model_to_opt = trained_models[best_yolo_name]
        yolo_cfg = next((cfg for cfg in CONFIG['yolo_models'] if cfg['name'] == best_yolo_name), None)

    # A. Export pour inférence optimisée (ONNX)
    print("📦 Export vers ONNX pour accélération de l'inférence...")
    try:
        model_to_opt.export(format='onnx', imgsz=CONFIG['img_size'][0])
        print("✅ Export ONNX terminé avec succès.")
    except Exception as e:
        print(f"⚠️ Erreur Export: {e}")

    # B. Optimisation des Hyperparamètres (Exemple rapide)
    print("🧬 Lancement d'un tuning rapide des hyperparamètres (5 epochs, 5 itérations)...")
    try:
        if yolo_cfg:
            model_to_opt.tune(data=yolo_cfg['data'], epochs=5, iterations=5, imgsz=CONFIG['img_size'][0], plots=False, verbose=False)
            print("✅ Tuning terminé.")
    except Exception as e:
        print(f"⚠️ Erreur Tuning: {e}")

    # C. Sauvegarde du modèle final
    print(f"💾 Sauvegarde du modèle final {best_yolo_name}...")
    try:
        # Copie du meilleur checkpoint de l'entraînement
        source_path = os.path.join(CONFIG['yolo_runs_dir'], best_yolo_name, 'weights', 'best.pt')
        dest_path = os.path.join(CONFIG['output_dir'], f'final_best_{best_yolo_name}.pt')
        if os.path.exists(source_path):
            import shutil
            shutil.copy(source_path, dest_path)
            print(f"✅ Modèle final copié vers : {dest_path}")
    except Exception as e:
        print(f"⚠️ Erreur Sauvegarde: {e}")

print(f"\n✅ Pipeline terminé avec succès (256x256 + CutMix).")

## 9. Chargement, Inférence et Analyse Approfondie
Dans cette section, nous chargeons le modèle YOLOv9 sauvegardé pour effectuer une inférence sur une image inédite. Nous affichons également la matrice de confusion générée lors de la validation pour analyser les erreurs de classification, ainsi que le tableau récapitulatif des métriques.

In [ ]:
# 8. Chargement du modèle final, Inférence et Matrice de Confusion
print("\n🔍 Chargement du modèle final et analyse...")

# Use the variable if defined, otherwise try to find it
if 'best_yolo_name' in locals():
    final_model_path = os.path.join(CONFIG['output_dir'], f'final_best_{best_yolo_name}.pt')
else:
    # Fallback search
    potential_files = glob.glob(os.path.join(CONFIG['output_dir'], 'final_best_*.pt'))
    final_model_path = potential_files[0] if potential_files else ""

if os.path.exists(final_model_path):
    # A. Chargement
    print(f"📥 Chargement de {final_model_path}...")
    loaded_model = YOLO(final_model_path)

    # B. Inférence sur des images de test (Original vs Prédiction)
    test_files = glob.glob(os.path.join(CONFIG['test_images_dir'], '*.*'))
    if test_files:
        print(f"🖼️ Visualisation des résultats sur {min(3, len(test_files))} images de test...")
        for img_path in test_files[:3]:
            # Image originale
            img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

            # Prédiction
            results = loaded_model.predict(img_path, imgsz=CONFIG['img_size'][0], conf=0.25, verbose=False)
            res_plotted = results[0].plot()
            res_plotted = cv2.cvtColor(res_plotted, cv2.COLOR_BGR2RGB)

            # Affichage côte à côte
            fig, axes = plt.subplots(1, 2, figsize=(15, 7))

            axes[0].imshow(img)
            axes[0].set_title(f"Original : {os.path.basename(img_path)}")
            axes[0].axis('off')

            axes[1].imshow(res_plotted)
            axes[1].set_title(f"Prédiction YOLOv9")
            axes[1].axis('off')

            plt.tight_layout()
            plt.show()

    # C. Matrice de Confusion et Métriques
    print("📉 Évaluation du modèle optimisé sur le jeu de validation...")
    # Reuse yolo_cfg from optimization block if available, or find it based on filename
    if 'yolo_cfg' in locals() and yolo_cfg:
        # Validation pour générer la matrice (plots=True est essentiel)
        metrics = loaded_model.val(data=yolo_cfg['data'], imgsz=CONFIG['img_size'][0], plots=True, verbose=False)

        print(f"\n📊 Métriques après optimisation ({best_yolo_name}):")
        print(f"   mAP50-95: {metrics.seg.map:.4f}")
        print(f"   mAP50:    {metrics.seg.map50:.4f}")

        # Recherche de l'image de la matrice de confusion
        cm_path = os.path.join(metrics.save_dir, 'confusion_matrix_normalized.png')
        if not os.path.exists(cm_path):
             cm_path = os.path.join(metrics.save_dir, 'confusion_matrix.png')

        if os.path.exists(cm_path):
            print(f"✅ Affichage de la matrice de confusion : {cm_path}")
            plt.figure(figsize=(12, 10))
            plt.imshow(cv2.cvtColor(cv2.imread(cm_path), cv2.COLOR_BGR2RGB))
            plt.title("Matrice de Confusion Normalisée (YOLOv9)")
            plt.axis('off')
            plt.show()
            
            # Sauvegarde séparée de la matrice de confusion optimisée
            import shutil
            shutil.copy(cm_path, os.path.join(CONFIG['output_dir'], 'optimized_confusion_matrix.png'))
        else:
            print("⚠️ Impossible de trouver l'image de la matrice de confusion.")
            
    # D. Sauvegarde des résultats optimisés séparés
    print("\n💾 Sauvegarde des outputs optimisés séparés...")
    
    # 1. Tableau de métriques optimisé
    opt_metrics_data = [{
        'Model': f'{best_yolo_name}_Optimized',
        'mIoU': metrics.seg.map,
        'mAP50': metrics.seg.map50,
        'Precision': metrics.seg.mp,
        'Recall': metrics.seg.mr
    }]
    opt_csv_path = os.path.join(CONFIG['output_dir'], 'optimized_model_results.csv')
    pd.DataFrame(opt_metrics_data).to_csv(opt_csv_path, index=False)
    print(f"   - Tableau de métriques : {opt_csv_path}")
    
    # 2. Images de prédiction optimisées
    if test_files:
        for img_path in test_files[:3]:
            results = loaded_model.predict(img_path, imgsz=CONFIG['img_size'][0], conf=0.25, verbose=False)
            res_plotted = results[0].plot()
            fname = os.path.basename(img_path)
            save_path = os.path.join(CONFIG['output_dir'], f"optimized_prediction_{fname}")
            cv2.imwrite(save_path, res_plotted)
            print(f"   - Image optimisée : {save_path}")

# D. Affichage du Tableau Récapitulatif (Comparaison Initiale)
csv_path = os.path.join(CONFIG['output_dir'], 'final_model_comparison_256.csv')
if os.path.exists(csv_path):
    print("\n📊 Rappel des Performances (Comparaison Initiale) :")
    df_results = pd.read_csv(csv_path)
    try:
        from IPython.display import display
        display(df_results)
    except ImportError:
        print(df_results.to_string(index=False))
else:
    print("⚠️ Pas de fichier de résultats trouvé.")

## 10. Conclusion

Ce projet a permis de comparer une approche de segmentation sémantique classique (U-Net) avec des modèles YOLO récents (v8, v9, v11) sur le dataset Cityscapes réduit.

**Points clés :**
- **Performance :** Les modèles YOLO, en particulier YOLOv9, offrent généralement un excellent compromis entre précision (mIoU) et vitesse d'inférence.
- **Robustesse :** L'utilisation de **CutMix** a permis d'améliorer la robustesse du modèle face aux occlusions partielles.
- **Optimisation :** L'export ONNX et le tuning des hyperparamètres montrent des pistes pour le déploiement en temps réel sur véhicule autonome.

**Perspectives :**
- Entraîner sur l'ensemble complet de Cityscapes.
- Tester des techniques d'augmentation plus poussées (Mosaic, MixUp).
- Intégrer un post-traitement (CRF) pour affiner les masques.